# Node2Vec algorithm

Paper: https://arxiv.org/pdf/1607.00653

## Abstract
Here we propose node2vec, an algorithmic framework for learning continuous feature representations for nodes in networks. In
node2vec, we learn a mapping of nodes to a low-dimensional space
of features that maximizes the likelihood of preserving network
neighborhoods of nodes. We define a flexible notion of a node’s
network neighborhood and design a biased random walk procedure,
which efficiently explores diverse neighborhoods. Our algorithm
generalizes prior work which is based on rigid notions of network
neighborhoods, and we argue that the added flexibility in exploring
neighborhoods is the key to learning richer representations

## Actual work 
Our experiments focus on two common prediction tasks in networks: a multi-label classification task, where every node is assigned one or more class labels and a link prediction task, where we
predict the existence of an edge given a pair of nodes. We contrast
the performance of node2vec with state-of-the-art feature learning
algorithms [24, 28]. We experiment with several real-world networks from diverse domains, such as social networks, information
networks, as well as networks from systems biology.

In [1]:
import os.path as osp
import sys

import matplotlib.pyplot as plt
import torch
from sklearn.manifold import TSNE

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec

In [2]:
path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
dataset = Planetoid(path, name='Cora')
data = dataset[0]

NameError: name '__file__' is not defined

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Node2Vec(
    data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,
    q=1.0,
    sparse=True,
).to(device)

num_workers = 4 if sys.platform == 'linux' else 0
loader = model.loader(batch_size=128, shuffle=True, num_workers=num_workers)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


In [ ]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(
        train_z=z[data.train_mask],
        train_y=data.y[data.train_mask],
        test_z=z[data.test_mask],
        test_y=data.y[data.test_mask],
        max_iter=150,
    )
    return acc


for epoch in range(1, 101):
    loss = train()
    acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {acc:.4f}')


@torch.no_grad()
def plot_points(colors):
    model.eval()
    z = model().cpu().numpy()
    z = TSNE(n_components=2).fit_transform(z)
    y = data.y.cpu().numpy()

    plt.figure(figsize=(8, 8))
    for i in range(dataset.num_classes):
        plt.scatter(z[y == i, 0], z[y == i, 1], s=20, color=colors[i])
    plt.axis('off')
    plt.show()


colors = [
    '#ffc0cb', '#bada55', '#008080', '#420420', '#7fe5f0', '#065535', '#ffd700'
]
plot_points(colors)

In [ ]:
import karateclub 

In [ ]:
# Implementation Example: Node2vec in PyTorch
# This example demonstrates a simple implementation of node2vec using PyTorch.

import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import random
import numpy as np

# Step 1: Construct a Graph for Node2vec
# --------------------------------------
# Create a homogeneous graph using NetworkX
G = nx.fast_gnp_random_graph(n=100, p=0.1)  # Random graph with 100 nodes

# Define custom random walk function for node2vec
# This function will generate sequences of nodes based on the random walk approach

def generate_node2vec_walks(graph, num_walks, walk_length):
    walks = []
    nodes = list(graph.nodes())
    for _ in range(num_walks):
        for node in nodes:
            walk = [node]
            for _ in range(walk_length - 1):
                neighbors = list(graph.neighbors(walk[-1]))
                if neighbors:
                    walk.append(random.choice(neighbors))
                else:
                    break
            walks.append(walk)
    return walks

# Generate random walks for training
num_walks = 10
walk_length = 5
generated_walks = generate_node2vec_walks(G, num_walks, walk_length)

# Train Word2Vec model on generated walks (node2vec)
walks = [list(map(str, walk)) for walk in generated_walks]
node2vec_model = Word2Vec(walks, vector_size=128, window=5, min_count=0, sg=1, workers=4, epochs=10)

# Step 2: Define Skip-gram Model in PyTorch
# -----------------------------------------
class SkipGramModel(nn.Module):
    def __init__(self, num_nodes, embedding_dim):
        super(SkipGramModel, self).__init__()
        self.node_embeddings = nn.Embedding(num_nodes, embedding_dim)
    
    def forward(self, center_node, context_node):
        center_embed = self.node_embeddings(center_node)
        context_embed = self.node_embeddings(context_node)
        score = torch.matmul(center_embed, context_embed.t())
        return score

# Step 3: Measuring Effectiveness
# ----------------------------------
# Define labels for nodes for a classification task (e.g., community detection or node classification)
labels = np.random.randint(0, 2, size=100)  # Random binary labels for demonstration purposes

# Split data into train and test sets
X = [node2vec_model.wv[str(node)] for node in range(100)]
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a simple classifier (e.g., Logistic Regression) to evaluate embeddings
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict and evaluate performance
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Node2Vec Embedding for Node 0:")
print(node2vec_model.wv['0'])
print(f"Accuracy of Node2Vec Embeddings: {accuracy}")
print(f"F1 Score of Node2Vec Embeddings: {f1}")